In [1]:
from theano.sandbox import cuda

In [34]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [35]:
model_path = 'courses/deeplearning1/nbs/'

In [36]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [37]:
idx_arr = sorted(idx,key = idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [38]:
idx2word = {v: k for k,v in idx.iteritems()}

In [39]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [40]:
len(x_train)

25000

In [41]:
', '.join(map(str,x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [42]:
idx2word[23022]

'bromwell'

In [43]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [44]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [45]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [46]:
lens = np.array(map(len,trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [47]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen = seq_len, value = 0)
test = sequence.pad_sequences(test,maxlen= seq_len, value = 0)

In [48]:
trn.shape

(25000, 500)

In [49]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])

In [50]:
model.compile(loss= 'binary_crossentropy', optimizer = Adam(), metrics = ['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 500, 32)       160000      embedding_input_3[0][0]          
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 16000)         0           embedding_3[0][0]                
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 100)           1600100     flatten_3[0][0]                  
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 100)           0           dense_5[0][0]                    
___________________________________________________________________________________________

In [51]:
model.fit(trn,labels_train, validation_data=(test, labels_test), nb_epoch =2, batch_size = 64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 25s - loss: 0.4757 - acc: 0.7398 - val_loss: 0.2895 - val_acc: 0.8779

In [52]:
conv1 = Sequential([
    Embedding(vocab_size,32,input_length=seq_len,dropout=0.2),
    Dropout(0.2),
    Convolution1D(64,5,border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [53]:
conv1.compile(loss='binary_crossentropy', optimizer =Adam(), metrics = ['accuracy'])

In [54]:


conv1.fit(trn,labels_train, validation_data=(test,labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 233s - loss: 0.4840 - acc: 0.7457 - val_loss: 0.3045 - val_acc: 0.8784

In [55]:
conv1.save_weights(model_path+'conv1.h5')

In [56]:
conv1.load_weights(model_path+'conv1.h5')